# Profiling Code Execution with cProfile


In [10]:
import cProfile
import pandas as pd

from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader

# Custom import of local file "decode"
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

# Custom import of local file "encode"
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

In [11]:
file = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav"
compressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.brainwire"

In [12]:
with cProfile.Profile() as pr:
    parser = encode.initialize_argument_parser()
    args = parser.parse_args([file, compressed_file_path, "-m=q", "-v"])
    encode.main(args)

df = pd.DataFrame(
    pr.getstats(), columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"]
)


Executed Line: create_huffman_encoded_file...
Time Δ Nanoseconds: 2513447000
Time Δ Microseconds: 2513447.0
Time Δ Milliseconds: 2513.447
Time Δ Seconds: 2.513447


Original File Size: 197526
Compressed File Size: 119487
Percent of Compression: 39.51%
Percent of Compressed File Size Relative to Required File Size: 12106.079%


In [17]:
def print_full(data_frame):
    """This function will display all rows of the dataframe.

    Args:
        data_frame (DataFrame): This is a pandas dataframe of
                                arbitrary length.
    """
    pd.set_option("display.max_rows", len(data_frame))
    pd.set_option("display.width", 1000)
    print(data_frame)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.width")

In [29]:
df[df["cumtime"] == df["cumtime"].max()]

,func,ncalls,ccalls,tottime,cumtime,callers
165,"<code object convertHexToBit at 0x12f275080, f...",1,0,1.985753,1.985617,"[(<built-in method builtins.len>, 3, 0, 1.126e..."


In [35]:
df.sort_values("tottime", ascending=False)

,func,ncalls,ccalls,tottime,cumtime,callers
114,"<code object main at 0x10540d270, file ""../../...",1,0,2.515429e+00,4.887400e-05,"[(<built-in method time.time_ns>, 2, 0, 1.209e..."
163,<code object create_huffman_encoded_file at 0x...,1,0,2.487523e+00,4.208300e-05,[(<code object huffman_encoding at 0x12f823600...
168,"<code object huffman_encoding at 0x12f823600, ...",1,0,2.069019e+00,1.125025e-02,"[(<built-in method builtins.len>, 66, 0, 4.794..."
165,"<code object convertHexToBit at 0x12f275080, f...",1,0,1.985753e+00,1.985617e+00,"[(<built-in method builtins.len>, 3, 0, 1.126e..."
160,<code object create_byte_string at 0x12a1e2c00...,1,0,4.184620e-01,4.125393e-01,"[(<method 'append' of 'list' objects>, 6, 0, 7..."
...,...,...,...,...,...,...
91,"<code object <listcomp> at 0x1042bac30, file ""...",1,0,2.090000e-07,2.090000e-07,None
21,<method 'rstrip' of 'str' objects>,1,0,2.080000e-07,2.080000e-07,None
38,<method 'append' of 'collections.deque' objects>,1,0,1.670000e-07,1.670000e-07,None
69,"<code object is_set at 0x103445070, file ""/opt...",1,0,1.670000e-07,1.670000e-07,None


In [33]:
print_full(df.sort_values("cumtime", ascending=False))

                                                  func  ncalls  ccalls       tottime       cumtime                                            callers
26                     <built-in method builtins.iter>       1       0  1.250000e-07  1.250000e-07                                               None
69   <code object is_set at 0x103445070, file "/opt...       1       0  1.670000e-07  1.670000e-07                                               None
38    <method 'append' of 'collections.deque' objects>       1       0  1.670000e-07  1.670000e-07                                               None
21                  <method 'rstrip' of 'str' objects>       1       0  2.080000e-07  2.080000e-07                                               None
91   <code object <listcomp> at 0x1042bac30, file "...       1       0  2.090000e-07  2.090000e-07                                               None
2                  <method 'extend' of 'list' objects>       2       0  2.920000e-07  2.920000e-07  